#### Import dependencies:

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
pandas version: 1.1.3
matplotlib version: 3.3.2
NumPy version: 1.19.5
SciPy version: 1.5.2
IPython version: 7.19.0
scikit-learn version: 0.23.2
-------------------------


#### Download data file if it has not been downloaded already:

In [2]:
#url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_src = 'https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD-summary-dashboard-123.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

File NYPD-summary-dashboard-123.csv has already been downloaded


#### Open a stream to the data file so we don't have to load the whole data set into main memory

In [3]:
from openclean.pipeline import stream
ds_full = stream(fn_dst)

In [4]:
print(f'The dataset contains {ds_full.count():,} rows.')

The dataset contains 115,299 rows.


In [5]:
ds_full.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,234233843,09/29/2021,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,B,42,0,25-44,M,BLACK,1009231,240290,40.82618898100003,-73.90973778899996,POINT (-73.90973778899996 40.82618898100003)
1,234129823,09/27/2021,157,RAPE 1,104,RAPE,PL 1303501,F,K,77,0,25-44,M,BLACK,1003606,185050,40.67458330800008,-73.93022154099998,POINT (-73.93022154099998 40.67458330800008)
2,234040747,09/25/2021,109,"ASSAULT 2,1,UNCLASSIFIED",106,FELONY ASSAULT,PL 1200501,F,Q,101,0,25-44,M,BLACK,1049232,159210,40.60344094100003,-73.76598558899997,POINT (-73.76598558899997 40.60344094100003)
3,234047720,09/25/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,M,BLACK,1006537,244511,40.83778161800007,-73.91945797099999,POINT (-73.91945797099999 40.83778161800007)
4,234042526,09/25/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,M,BLACK,1007418,243859,40.83598980000005,-73.91627635999998,POINT (-73.91627635999998 40.83598980000005)
5,234006839,09/24/2021,582,,,,PL 2225501,F,M,23,0,25-44,M,BLACK,998659,228177,40.79296572700008,-73.94796350499998,POINT (-73.94796350499998 40.79296572700008)
6,233952333,09/23/2021,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1600500,F,K,75,0,18-24,M,BLACK,1017119,183909,40.67141166300007,-73.88151172399995,POINT (-73.88151172399995 40.67141166300007)
7,233878488,09/21/2021,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211300,F,B,40,0,25-44,M,WHITE,1010679,234533,40.81038342800008,-73.90452841699994,POINT (-73.90452841699994 40.81038342800008)
8,233784866,09/19/2021,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,44,0,25-44,F,BLACK,1004489,237844,40.81948754300004,-73.926879392,POINT (-73.926879392 40.81948754300004)
9,233647454,09/16/2021,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1601503,F,K,61,0,18-24,M,BLACK,995834,161062,40.60875497300003,-73.95828138999997,POINT (-73.95828138999997 40.60875497300003)


#### Use openclean to profile the data set:

In [6]:
from openclean.profiling.column import DefaultColumnProfiler

#profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

In [7]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,115299,0,115299,1.000000,16.815020
ARREST_DATE,115299,0,273,0.002368,8.047698
PD_CD,115299,21,252,0.002186,5.401154
PD_DESC,115299,61,233,0.002022,5.366524
KY_CD,115299,61,66,0.000573,4.534242
OFNS_DESC,115299,61,63,0.000547,4.339051
LAW_CODE,115299,0,922,0.007997,6.328932
LAW_CAT_CD,115299,1078,4,0.000035,1.032090
ARREST_BORO,115299,0,5,0.000043,2.172560
ARREST_PRECINCT,115299,0,77,0.000668,6.119069


### Field Name & Description:
    'CMPLNT_NUM' (int)                 Complaint Number
    'CMPLNT_FR_DT' (date)              Complaint From Date
    'CMPLNT_FR_TM' (date)              Complaint From Time
    'CMPLNT_TO_DT' (date)              Complaint To Date
    'CMPLNT_TO_TM' (date)              Complaint To Time
    'ADDR_PCT_CD' (int)                Code of Precinct in which the Incident Occured
    'RPT_DT' (date)                    Report Date
    'KY_CD' (int)                      "Key Code": Offense Classification Code (3 digits)
    'OFNS_DESC' (str)                  Offense Description
    'PD_CD' (int)                      PD Code of Offense. More granular than Key Code
    'PD_DESC' (str)                    PD Description of Offense.
    'CRM_ATPT_CPTD_CD' (str)           Whether Crime was Atempted or Completed (values: 'COMPLETED', 'ATTEMPTED')
    'LAW_CAT_CD' (str)                 Level of Offense (values: 'FELONY', 'VIOLATION', 'MISDEMEANOR')
    'BORO_NM' (str)                    Name of Borough in which Incident Occurred
    'LOC_OF_OCCUR_DESC' (str)          Description of where the incident occurred with respect to the premises
                                       (values:'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF')
    'PREM_TYP_DESC' (str)              Description of the type of premises in which the Incident Occurred
    'JURIS_DESC' (str)                 Description of Jurisdiction in which Incident Occurred
    'JURISDICTION_CODE' (int)          Jurisdiction Code
    'PARKS_NM' (str)                   Name of Park in which Incident Occurred, if Applicable
    'HADEVELOPT' (str)                 Name of NYCHA Housing Development in which Incident Occurred, if Applicable
    'HOUSING_PSA' (int)                Housing PSA
    'X_COORD_CD' (int)                 X-coordinate, New York State Plane Coordinate System
    'Y_COORD_CD' (int)                 Y-coordinate, New York State Plane Coordinate System
    'SUSP_AGE_GROUP' (int)             Age Group of Suspect
    'SUSP_RACE' (str)                  Race of Suspect
    'SUSP_SEX' (str)                   Sex of Suspect
    'TRANSIT_DISTRICT' (int)           Transit-District code
    'Latitude' (float)                 Global Latitude of Location where Incident Occurred
    'Longitude' (float)                Global Longitude of Location where Incident Occured
    'Lat_Lon' (str)                    'Latitude' and 'Longitude' together
    'PATROL_BORO' (str)                Patrol Borough
    'STATION_NAME' (str)               Station Name
    'VIC_AGE_GROUP' (int)              Age Group of Victim
    'VIC_RACE' (str)                   Race of Victim
    'VIC_SEX' (str)                    Sex of Victim
    
    
    (Note: some field descriptions were taken from https://www1.nyc.gov/assets/nypd/downloads/pdf/analysis_and_planning/incident_level_data_footnotes.pdf)
   

## Profiling & Cleaning of each field in the data set

In [9]:
profiles.minmax('ARREST_KEY')

,min,max
int,222471128,234352983


In [10]:
profiles.minmax('ARREST_DATE')

,min,max
date,2021-01-01,2021-09-30


In [11]:
from datetime import datetime
#datetime.strptime('2014-12-04', '%Y-%m-%d').date()


def validate_date(sourceValue):
    dummy_date = datetime.strptime("01/01/1000", '%m/%d/%Y').date()
    
    try:
        source_date = sourceValue
        start_date = datetime.strptime("01/01/2021", '%m/%d/%Y').date()
        end_date = datetime.strptime("09/31/2021", '%m/%d/%Y').date()
    
        if source_date == '':
            source_date = dummy_date
        
        source_date = datetime.strptime(sourceValue, '%m/%d/%Y').date()
        
        if source_date >= start_date and source_date <= end_date:
            return source_date
        else:
            return dummy_date
    except:
        return dummy_date
    

ds_full = ds_full.update('ARREST_DATE', validate_date)

In [12]:
profiles.minmax('ARREST_PRECINCT')

,min,max
int,1,123


In [13]:
profiles.column('ARREST_PRECINCT').get('topValues')

[('14', 3301),
 ('44', 3205),
 ('75', 3178),
 ('40', 3040),
 ('47', 2640),
 ('114', 2568),
 ('43', 2509),
 ('103', 2491),
 ('46', 2438),
 ('52', 2399)]

In [14]:
profiles.minmax('KY_CD')

,min,max
int,101,995


In [15]:
profiles.column('KY_CD').get('topValues')

[('344', 19937),
 ('106', 11472),
 ('341', 9132),
 ('126', 7577),
 ('105', 5706),
 ('109', 4760),
 ('117', 4260),
 ('118', 4135),
 ('348', 3995),
 ('235', 3903)]

In [16]:
profiles.column('OFNS_DESC').get('topValues')

[('ASSAULT 3 & RELATED OFFENSES', 19937),
 ('FELONY ASSAULT', 11472),
 ('PETIT LARCENY', 9132),
 ('DANGEROUS DRUGS', 8163),
 ('MISCELLANEOUS PENAL LAW', 7629),
 ('CRIMINAL MISCHIEF & RELATED OF', 7007),
 ('ROBBERY', 5706),
 ('DANGEROUS WEAPONS', 5460),
 ('GRAND LARCENY', 4760),
 ('VEHICLE AND TRAFFIC LAWS', 3995)]

In [17]:
ds_full.distinct('OFNS_DESC')

Counter({'FELONY ASSAULT': 11472,
         'RAPE': 459,
         'ASSAULT 3 & RELATED OFFENSES': 19937,
         '': 61,
         'ROBBERY': 5706,
         'BURGLARY': 3507,
         'SEX CRIMES': 2600,
         'ARSON': 86,
         'OFFENSES AGAINST PUBLIC ADMINI': 3462,
         'DANGEROUS DRUGS': 8163,
         'MISCELLANEOUS PENAL LAW': 7629,
         'FORGERY': 2487,
         'PETIT LARCENY': 9132,
         'POSSESSION OF STOLEN PROPERTY': 1359,
         'VEHICLE AND TRAFFIC LAWS': 3995,
         'INTOXICATED/IMPAIRED DRIVING': 284,
         'NYS LAWS-UNCLASSIFIED FELONY': 566,
         'CRIMINAL MISCHIEF & RELATED OF': 7007,
         'DANGEROUS WEAPONS': 5460,
         'OFFENSES INVOLVING FRAUD': 2966,
         'GRAND LARCENY': 4760,
         'OTHER TRAFFIC INFRACTION': 1209,
         'OFF. AGNST PUB ORD SENSBLTY &': 2548,
         'MURDER & NON-NEGL. MANSLAUGHTE': 1157,
         'ALCOHOLIC BEVERAGE CONTROL LAW': 279,
         'GAMBLING': 244,
         'FOR OTHER AUTHORITIES': 1

In [18]:
profiles.minmax('PD_CD')

,min,max
int,0,997


In [19]:
# Print the most frequent values in column ''

profiles.column('PD_CD').get('topValues')

[('101', 14678),
 ('339', 9110),
 ('109', 8591),
 ('397', 5678),
 ('779', 5256),
 ('439', 4100),
 ('922', 3728),
 ('792', 3461),
 ('113', 3296),
 ('244', 3208)]

In [21]:
# Print the most frequent values in column 'PD_DESC'

profiles.column('PD_DESC').get('topValues')

[('ASSAULT 3', 14678),
 ('LARCENY,PETIT FROM OPEN AREAS,', 9110),
 ('ASSAULT 2,1,UNCLASSIFIED', 8591),
 ('ROBBERY,OPEN AREA UNCLASSIFIED', 5678),
 ('PUBLIC ADMINISTRATION,UNCLASSI', 5256),
 ('LARCENY,GRAND FROM OPEN AREAS, UNATTENDED', 4100),
 ('TRAFFIC,UNCLASSIFIED MISDEMEAN', 3728),
 ('CONTROLLED SUBSTANCE, POSSESSI', 3463),
 ('WEAPONS POSSESSION 1 & 2', 3461),
 ('MENACING,UNCLASSIFIED', 3296)]

In [22]:
ds_full.distinct('PD_DESC')

Counter({'STRANGULATION 1ST': 2106,
         'RAPE 1': 258,
         'ASSAULT 2,1,UNCLASSIFIED': 8591,
         'ASSAULT 3': 14678,
         '': 61,
         'ROBBERY,OPEN AREA UNCLASSIFIED': 5678,
         'BURGLARY,UNCLASSIFIED,UNKNOWN': 3208,
         'SODOMY 2': 9,
         'RAPE 3': 149,
         'ARSON 2,3,4': 81,
         'CONTEMPT,CRIMINAL': 1815,
         'CONTROLLED SUBSTANCE,SALE 3': 937,
         'PUBLIC ADMINISTRATION,UNCLASSI': 5256,
         'CHILD, ENDANGERING WELFARE': 1351,
         'FORGERY,ETC.,UNCLASSIFIED-FELO': 2444,
         'CONSPIRACY 2, 1': 103,
         'RAPE 2': 52,
         'SEXUAL ABUSE': 258,
         'LARCENY,PETIT FROM OPEN AREAS,': 9110,
         'SEXUAL ABUSE 3,2': 716,
         'RESISTING ARREST': 879,
         'STOLEN PROPERTY 3,POSSESSION': 534,
         'TRAFFIC,UNCLASSIFIED MISDEMEAN': 3728,
         'CONTROLLED SUBSTANCE, POSSESSI': 3463,
         'INTOXICATED DRIVING,ALCOHOL': 1919,
         'NY STATE LAWS,UNCLASSIFIED FEL': 558,
         'CRI

In [23]:
ds_full.distinct('LAW_CAT_CD')

Counter({'F': 52578, 'M': 61204, '': 1078, 'I': 169, 'V': 270})

In [24]:
def validate_law_cat_cd(sourceValue):
    UNKNOWN = "UNKNOWN"
    
    try:
        values = ["F", "M", "V", "I"] 
        
        if sourceValue in values:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('LAW_CAT_CD', validate_law_cat_cd)

In [25]:
ds_full.distinct('LAW_CAT_CD')

Counter({'F': 52578, 'M': 61204, 'UNKNOWN': 1078, 'I': 169, 'V': 270})

In [27]:
ds_full.distinct('ARREST_BORO')

Counter({'B': 25312, 'K': 31004, 'Q': 24654, 'M': 28948, 'S': 5381})

In [28]:
profiles.column('JURISDICTION_CODE').get('topValues')

[('0', 105588),
 ('2', 3803),
 ('1', 3162),
 ('3', 1034),
 ('97', 493),
 ('15', 262),
 ('72', 189),
 ('11', 187),
 ('73', 142),
 ('71', 125)]

In [30]:
ds_full.distinct('JURISDICTION_CODE')

Counter({'0': 105588,
         '1': 3162,
         '71': 125,
         '2': 3803,
         '3': 1034,
         '15': 262,
         '97': 493,
         '72': 189,
         '73': 142,
         '7': 83,
         '11': 187,
         '14': 89,
         '87': 24,
         '4': 69,
         '12': 1,
         '85': 8,
         '13': 11,
         '6': 10,
         '88': 8,
         '74': 3,
         '16': 2,
         '8': 1,
         '69': 3,
         '76': 1,
         '9': 1})

In [31]:
profiles.minmax('X_COORD_CD')

,min,max
int,914214,1066556


In [32]:
profiles.minmax('Y_COORD_CD')

,min,max
int,121312,271820


In [33]:
profiles.column('AGE_GROUP').get('topValues')

[('25-44', 65846),
 ('45-64', 22438),
 ('18-24', 21931),
 ('<18', 3408),
 ('65+', 1676)]

In [34]:
ds_full.distinct('AGE_GROUP')

Counter({'25-44': 65846,
         '18-24': 21931,
         '45-64': 22438,
         '<18': 3408,
         '65+': 1676})

In [35]:
profiles.column('PERP_RACE').get('topValues')

[('BLACK', 57011),
 ('WHITE HISPANIC', 28355),
 ('WHITE', 12586),
 ('BLACK HISPANIC', 10208),
 ('ASIAN / PACIFIC ISLANDER', 6389),
 ('UNKNOWN', 431),
 ('AMERICAN INDIAN/ALASKAN NATIVE', 319)]

In [36]:
ds_full.distinct('PERP_RACE')

Counter({'BLACK': 57011,
         'WHITE': 12586,
         'WHITE HISPANIC': 28355,
         'BLACK HISPANIC': 10208,
         'ASIAN / PACIFIC ISLANDER': 6389,
         'AMERICAN INDIAN/ALASKAN NATIVE': 319,
         'UNKNOWN': 431})

In [37]:
profiles.column('PERP_SEX').get('topValues')

[('M', 95849), ('F', 19450)]

In [38]:
ds_full.distinct('PERP_SEX')

Counter({'M': 95849, 'F': 19450})

In [39]:
profiles.minmax('Latitude')

,min,max
float,40.499401,40.912723


In [40]:
def validate_lat(sourceValue):
    UNKNOWN = -999
    
    try:
        if sourceValue >= 40 and sourceValue <= 41:  
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('Latitude', validate_lat)

In [41]:
profiles.minmax('Longitude')

,min,max
float,-74.251844,-73.703


In [42]:
def validate_long(sourceValue):
    UNKNOWN = -999
    
    try:
        if sourceValue >= -74 and sourceValue <= -73:  
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('Latitude', validate_long)

In [43]:
profiles.minmax('New Georeferenced Column')

,min,max
str,POINT (-73.70300043899994 40.73765300700006),POINT (-74.25184392199996 40.50426856400002)


In [50]:
SELECTED_COLUMNS = [\
    'ARREST_KEY',\
    'ARREST_DATE',\
    'PD_CD',\
    'PD_DESC',\
    'KY_CD',\
    'OFNS_DESC',\
    'LAW_CODE',\
    'LAW_CAT_CD',\
    'ARREST_BORO',\
    'ARREST_PRECINCT',\
    'JURISDICTION_CODE',\
    'AGE_GROUP',\
    'PERP_SEX',\
    'PERP_RACE',\
    'X_COORD_CD',\
    'Y_COORD_CD',\
    'Latitude',\
    'Longitude',\
    'New Georeferenced Column',\
]
ds_full = ds_full.select(SELECTED_COLUMNS)

In [51]:
ds_full

# VALIDATING DATA AFTER CLEANING

In [52]:
data_df = pd.read_csv("NYPD-summary-dashboard-123.csv")

In [53]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115299 entries, 0 to 115298
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ARREST_KEY                115299 non-null  int64  
 1   ARREST_DATE               115299 non-null  object 
 2   PD_CD                     115278 non-null  float64
 3   PD_DESC                   115238 non-null  object 
 4   KY_CD                     115238 non-null  float64
 5   OFNS_DESC                 115238 non-null  object 
 6   LAW_CODE                  115299 non-null  object 
 7   LAW_CAT_CD                114221 non-null  object 
 8   ARREST_BORO               115299 non-null  object 
 9   ARREST_PRECINCT           115299 non-null  int64  
 10  JURISDICTION_CODE         115299 non-null  int64  
 11  AGE_GROUP                 115299 non-null  object 
 12  PERP_SEX                  115299 non-null  object 
 13  PERP_RACE                 115299 non-null  o

In [54]:
ds_full_jc = stream(data_df)
profiles = ds_full_jc.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,115299,0,115299,1.000000,16.815020
ARREST_DATE,115299,0,273,0.002368,8.047698
PD_CD,115299,21,252,0.002186,5.401154
PD_DESC,115299,61,233,0.002022,5.366524
KY_CD,115299,61,66,0.000573,4.534242
OFNS_DESC,115299,61,63,0.000547,4.339051
LAW_CODE,115299,0,922,0.007997,6.328932
LAW_CAT_CD,115299,1078,4,0.000035,1.032090
ARREST_BORO,115299,0,5,0.000043,2.172560
ARREST_PRECINCT,115299,0,77,0.000668,6.119069


In [55]:
data_df = data_df.dropna()

In [56]:
ds_full_jc = stream(data_df)
profiles = ds_full_jc.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,114160,0,114160,1.000000,16.800698
ARREST_DATE,114160,0,273,0.002391,8.048743
PD_CD,114160,0,237,0.002076,5.366757
PD_DESC,114160,0,228,0.001997,5.335263
KY_CD,114160,0,65,0.000569,4.499853
OFNS_DESC,114160,0,62,0.000543,4.302819
LAW_CODE,114160,0,898,0.007866,6.301579
LAW_CAT_CD,114160,0,4,0.000035,1.032110
ARREST_BORO,114160,0,5,0.000044,2.173442
ARREST_PRECINCT,114160,0,77,0.000674,6.117894


In [57]:
data_df.to_csv('filtered8.csv')